In [2]:
import pandas as pd

from sqlalchemy import create_engine

In [3]:
cnx = create_engine('postgresql://ubuntu@18.191.26.249:5432/covid19mx')

In [25]:
population_density = pd.read_html("https://en.wikipedia.org/wiki/List_of_Mexican_states_by_population_density")
population_density = population_density[1]
population_density['State'] = population_density['State'].str[3:].str.strip()
population_density.rename(columns = {
        population_density.columns[1]: 'Population' ,
        population_density.columns[2]: 'Area' ,
        population_density.columns[3]: 'PopulationDensity' ,
    },inplace=True)
population_density.State = population_density.State.replace("United Mexican States (México)", "United Mexican States")
population_density.State = population_density.State.replace("State of Mexico", "Mexico State")
population_density.to_sql('pop_density', con=cnx, if_exists='replace')
population_density.head()

,State,Population,Area,PopulationDensity
0,Mexico City,8851080,1485,5920.0
1,Mexico State,15175862,22357,679.0
2,Morelos,1777227,4893,364.0
3,Tlaxcala,1169936,3991,293.0
4,Aguascalientes,1184996,5618,211.0


In [24]:
Human_Development_Index = pd.read_html("https://en.wikipedia.org/wiki/List_of_Mexican_states_by_Human_Development_Index")
Human_Development_Index = Human_Development_Index[0]
Human_Development_Index.columns = Human_Development_Index.columns.droplevel()
Human_Development_Index = Human_Development_Index.drop(columns = ["Unnamed: 6_level_1","Comparable Country (2017)","Comparable Country (2010)"])
Human_Development_Index[['Rank','2017','2010']] = Human_Development_Index[['Rank','2017','2010']].apply(pd.to_numeric, errors='coerce')
Human_Development_Index = Human_Development_Index.dropna()

Human_Development_Index["Federal Entity"] = Human_Development_Index["Federal Entity"].replace("Mexico City", "CIUDAD DE MÉXICO")
Human_Development_Index["Federal Entity"] = Human_Development_Index["Federal Entity"].replace("Querétaro de Arteaga", "QUERÉTARO")
Human_Development_Index["Federal Entity"] = Human_Development_Index["Federal Entity"].replace("Veracruz", "VERACRUZ DE IGNACIO DE LA LLAVE")
Human_Development_Index["Federal Entity"] = Human_Development_Index["Federal Entity"].replace("State of Mexico", "MÉXICO")
Human_Development_Index = Human_Development_Index.append(pd.DataFrame([["ESTADOS UNIDOS MEXICANOS",0.774]],columns=['Federal Entity','2017'])) 

Human_Development_Index.to_sql('HDI', con=cnx, if_exists='replace')
Human_Development_Index.head()

,Rank,Federal Entity,2017,2010
1,1.0,CIUDAD DE MÉXICO,0.834,0.818
2,2.0,Baja California,0.809,0.784
3,3.0,Nuevo León,0.807,0.788
4,4.0,Baja California Sur,0.805,0.771
5,5.0,Sonora,0.804,0.782


In [13]:
GDP_per_capita = pd.read_html("https://en.wikipedia.org/wiki/List_of_Mexican_states_by_GDP_per_capita")
GDP_per_capita = GDP_per_capita[0]
GDP_per_capita.columns = GDP_per_capita.columns.droplevel()
GDP_per_capita = GDP_per_capita.drop(columns = ["Unnamed: 4_level_1","GDP per capita (USD PPP)"])
GDP_per_capita.rename(columns = {
        GDP_per_capita.columns[2]: 'perCapita' ,
    },inplace=True)
GDP_per_capita.State = GDP_per_capita.State.replace("Ciudad de México", "Mexico City")
GDP_per_capita.to_sql('GDP_per_cap', con=cnx, if_exists='replace')
GDP_per_capita.head()

,Rank,State,perCapita
0,1,Campeche,474486
1,2,Mexico City,386135
2,3,Nuevo León,285709
3,4,Coahuila,245682
4,5,Sonora,234200


In [118]:
GDP = pd.read_html("https://en.wikipedia.org/wiki/List_of_Mexican_states_by_GDP")
GDP = GDP[0]
GDP.columns = GDP.columns.droplevel()
GDP = GDP.drop(columns = ["Unnamed: 5_level_1","Percent","GDP (millions of USD)"])
GDP.rename(columns = {
        GDP.columns[2]: 'GDP' ,
    },inplace=True)
GDP.to_sql('GDP', con=cnx, if_exists='replace')
GDP.head()

,Rank,State,GDP
0,-,United Mexican States,13401295
1,1,Mexico City,2254840
2,2,Mexico State,1209424
3,3,Nuevo León,999407
4,4,Jalisco,850237


In [28]:
poverty_rate = pd.read_html("https://en.wikipedia.org/wiki/List_of_Mexican_states_by_poverty_rate")
poverty_rate = poverty_rate[0]
poverty_rate['Poverty Rate (2012) [1]'] = poverty_rate['Poverty Rate (2012) [1]'].str.replace("%",'')
poverty_rate['Poverty Rate (2010) [2]'] = poverty_rate['Poverty Rate (2010) [2]'].str.replace("%",'')
poverty_rate.rename(columns = {
        poverty_rate.columns[2]: 'PovertyRate' ,
        poverty_rate.columns[3]: 'PovertyRate2010'
    },inplace=True)
poverty_rate.State = poverty_rate.State.replace("México", "Mexico State")
poverty_rate = poverty_rate.append(pd.DataFrame([["United Mexican States",42.0]],columns=['State','PovertyRate'])) 

poverty_rate.to_sql('poverty_rate', con=cnx, if_exists='replace')
poverty_rate.head()

,Rank,State,PovertyRate,PovertyRate2010
0,1,Chiapas,74.7,78.5
1,2,Guerrero,69.7,67.6
2,3,Puebla,64.5,61.5
3,4,Oaxaca,61.9,67.0
4,5,Tlaxcala,57.9,60.3
